# 멜론 월간차트 크롤링 (2021 01. ~ 02)
* 크롤링한 데이터 출처: https://www.melon.com/chart/search/index.htm (멜론 시대별차트 차트파인더)

In [1]:
import selenium
from selenium import webdriver
import time
import pandas as pd
from bs4 import BeautifulSoup
import requests
from itertools import repeat

In [2]:
# 크롬 드라이버 열기
driver = webdriver.Chrome('chromedriver.exe')
driver.implicitly_wait(5) # 대기 걸기
driver.maximize_window()


# 2021.01월 데이터 수집
* 트래픽에 부하를 줄 수 있으므로 1월과 2월을 나눠서 진행하였다

In [3]:
############### 1월 데이터 수집

    
# 크롤링으로 데이터 수집

# 각 컬럼별 데이터 리스트 선정 (15개)

year = [] # 연도
rank = [] # 순위
title = [] # 곡 명
singer= [] # 가수
album = [] # 앨범명
like_count = [] # 좋아요 수

date_of_issue = [] # 앨범 발매일
genre = [] # 장르
lyrics = [] # 가사
three = [] # 작사/작곡/편곡

fan_count = [] # 팬 수

member_count = [] # 멤버수
solor_group = [] # 솔로/그룹여부
gender = [] # 성별
debut_song = [] # 데뷔곡 명


# 1~50위곡 순위, 곡 제목, 가수, 앨범, 좋아요 수 수집
for i in range(50):
    # 드라이버가 해당 url 접속
    url = 'https://www.melon.com/chart/month/index.htm' # 멜론 월간 차트 url
    driver.get(url)


    # 국내 종합 클릭
    driver.find_element_by_xpath('/html/body/div/div[3]/div/div/div[2]/dl/dd[1]/ul/li[2]/a/span').click()


    # 월 고르기 버튼 클릭
    driver.find_element_by_xpath('/html/body/div/div[3]/div/div/div[3]/div/button').click()
    time.sleep(1)

    # 1월 클릭
    driver.find_element_by_xpath('/html/body/div/div[3]/div/div/div[3]/div/div/dl/dd[1]/ul/li[1]/a/span').click()
    time.sleep(1)
    
    year.append('2021.01')
    rank.append(driver.find_element_by_xpath(f'/html/body/div/div[3]/div/div/div[4]/form/div/table/tbody/tr[{i+1}]/td[2]/div/span[1]').text)
    title.append(driver.find_element_by_xpath(f'/html/body/div/div[3]/div/div/div[4]/form/div/table/tbody/tr[{i+1}]/td[6]/div/div/div[1]/span/a').text)
    singer.append(driver.find_element_by_xpath(f'/html/body/div/div[3]/div/div/div[4]/form/div/table/tbody/tr[{i+1}]/td[6]/div/div/div[2]/a[1]').text)
    album.append(driver.find_element_by_xpath(f'/html/body/div/div[3]/div/div/div[4]/form/div/table/tbody/tr[{i+1}]/td[7]/div/div/div/a').text)
    like_count.append(driver.find_element_by_xpath(f'/html/body/div/div[3]/div/div/div[4]/form/div/table/tbody/tr[{i+1}]/td[8]/div/button/span[2]').text)
    
    

    # 앨범명 클릭 페이지 (발매일, 장르, 가사, 작사/작곡/편곡)
    driver.find_element_by_xpath(f'/html/body/div/div[3]/div/div/div[4]/form/div/table/tbody/tr[{i+1}]/td[5]/div/a').click()
    time.sleep(2)
    date_of_issue.append(driver.find_element_by_xpath(f'/html/body/div[1]/div[3]/div/div/div/form/div/div/div[2]/div[2]/dl/dd[2]').text)
    genre.append(driver.find_element_by_xpath(f'/html/body/div[1]/div[3]/div/div/div/form/div/div/div[2]/div[2]/dl/dd[3]').text)
    lyrics.append(driver.find_element_by_xpath(f'/html/body/div[1]/div[3]/div/div/div/div[2]/div[2]/div').text)


    temp = driver.find_elements_by_xpath('/html/body/div[1]/div[3]/div/div/div/div[3]/ul')
    time.sleep(2)
    for i in range(len(temp)):
        three.append(temp[i].text)

    # 가수명 페이지 클릭 (팬 수를 수집하기 위함)
    driver.find_element_by_xpath('/html/body/div[1]/div[3]/div/div/div/form/div/div/div[2]/div[1]/div[2]/a/span[1]').click()
    time.sleep(2)
    fan_count.append(driver.find_element_by_id('d_like_count').text)

    # 상세정보 페이지 클릭 (멤버수,솔로/그룹,성별,데뷔곡을 수집하기 위함)
    try:
        driver.find_element_by_xpath('/html/body/div/div[3]/div/div/div[3]/ul/li[2]/a').click()
        time.sleep(2)
    except:
        driver.find_element_by_xpath('/html/body/div/div[3]/div/div/div[4]/ul/li[2]/a').click()
        time.sleep(2)

    try:
        member_count.append(len(driver.find_element_by_class_name('wrap_atistname').text.split(',')))
        solor_group.append("그룹")

    except:
        member_count.append(1)
        solor_group.append("솔로")

    mfm = driver.find_element_by_class_name('section_atistinfo03').text
    time.sleep(2)
    if mfm.find("혼성") >= 0:
        gender.append("혼성")
    elif mfm.find("남성") >= 0:
        gender.append("남성")
    else:
        gender.append("여성")

    try:
        debut_song.append(driver.find_element_by_class_name('debutsong').text)

    except:
        debut_song.append('')
    


In [6]:

#수집한 데이터들을 각 컬럼별로 리스트에 담아서 데이터프레임으로 만들기
df1 = pd.DataFrame({'연도' : year,
                   '순위': rank,
                   '제목': title,
                   '가수' : singer,
                   '앨범' : album,
                   '좋아요 수' : like_count,
                   '발매일' : date_of_issue,
                   '장르' : genre,
                   '가사' : lyrics,
                   '작사/작곡/편곡' : three,
                   '팬수' : fan_count,
                   '멤버수' : member_count,
                   '솔로/그룹' : solor_group,
                   '성별' : gender,
                   '데뷔곡' : debut_song
                  })
df1

,연도,순위,제목,가수,앨범,좋아요 수,발매일,장르,가사,작사/작곡/편곡,팬수,멤버수,솔로/그룹,성별,데뷔곡
0,2021.01,1,VVS (Feat. JUSTHIS) (Prod. GroovyRoom),미란이 (Mirani),쇼미더머니 9 Episode 1,"187,269",2020.11.21,랩/힙합,보여줘야겠어\n내가 망할 거랬던 놈들에게도\n내가 잘될 거라 했던 너에게도\n다 할...,저스디스 (JUSTHIS)\n작사\n그루비룸 (GroovyRoom)\n작사\n식케이...,"14,384",1,솔로,여성,
1,2021.01,2,밤하늘의 별을(2020),경서,밤하늘의 별을(2020),"157,568",2020.11.14,발라드,밤하늘의 별을 따서 너에게 줄래\n너는 내가 사랑하니까\n더 소중하니까\n오직 너 ...,강봄\n작사\n강봄\n작곡\n양정승\n편곡,"4,433",1,솔로,여성,
2,2021.01,3,Dynamite,방탄소년단,Dynamite (DayTime Version),"418,048",2020.08.24,댄스,Cos ah ah\nI’m in the stars tonight\nSo watch ...,David Stewart\n작사\nJessica Agombar\n작사\nDavid ...,"778,049",7,그룹,남성,데뷔곡\nNo More Dream\n방탄소년단
3,2021.01,4,잠이 오질 않네요,장범준,잠이 오질 않네요,"143,230",2020.10.24,발라드,당신은 날 설레게 만들어\n조용한 내 마음 자꾸만 춤추게 해\n얼마나 얼마나 날 떨...,장범준\n작사\n박경구\n작사\n장범준\n작곡\n장범준\n편곡\n전영호\n편곡\n황...,"103,946",1,솔로,남성,데뷔곡\n어려운 여자\n장범준
4,2021.01,5,Lovesick Girls,BLACKPINK,THE ALBUM,"194,970",2020.10.02,댄스,영원한 밤\n창문 없는 방에 우릴 가둔 love\nWhat can we say\n매...,TEDDY\n작사\nLØREN\n작사\n지수 (JISOO)\n작사\n제니 (JENN...,"209,935",4,그룹,여성,데뷔곡\n휘파람\nBLACKPINK
5,2021.01,6,취기를 빌려 (취향저격 그녀 X 산들),산들,취기를 빌려 (취향저격 그녀 X 산들),"195,841",2020.07.20,발라드,언제부턴가 불쑥\n내 습관이 돼버린 너\n혹시나 이런 맘이\n어쩌면 부담일까\n널 ...,새봄 (saevom)\n작사\n새봄 (saevom)\n작곡\n박민주\n편곡,"27,249",1,솔로,남성,데뷔곡\nO.K\n산들
6,2021.01,7,힘든 건 사랑이 아니다,임창정,힘든 건 사랑이 아니다,"106,204",2020.10.19,발라드,내가 널 떠났어야 했는데 왜 떠나야 하는지도 아는데\n어떤 아무 말도 아무것도 줄 ...,임창정\n작사\n임창정\n작곡\n멧돼지\n작곡\n늑대\n작곡\n멧돼지\n편곡\n늑대...,"95,456",1,솔로,남성,
7,2021.01,8,"내일이 오면 (Feat. 기리보이, BIG Naughty (서동현))",릴보이 (lIlBOI),쇼미더머니 9 Episode 3,"107,679",2020.12.05,랩/힙합,See I just wanna\nshare the vibe with ya\n우리가 ...,릴보이 (lIlBOI)\n작사\n기리보이\n작사\nBIG Naughty (서동현)\...,"24,953",1,솔로,남성,데뷔곡\n그룹\nOfficially Missing You\n긱스 (Geeks)
8,2021.01,9,Life Goes On,방탄소년단,BE,"221,803",2020.11.20,랩/힙합,어느 날 세상이 멈췄어\n아무런 예고도 하나 없이\n봄은 기다림을 몰라서\n눈치 없...,Pdogg\n작사\nRM\n작사\nRuuth\n작사\nChris James\n작사\...,"778,050",7,그룹,남성,데뷔곡\nNo More Dream\n방탄소년단
9,2021.01,10,오래된 노래,스탠딩 에그,오래된 노래,"230,801",2012.09.04,"인디음악, 포크/블루스",오래 전에 함께 듣던 노래가\n발걸음을 다시 멈춰서게 해\n이 거리에서\n너를 느낄...,Egg1\n작사\nEgg2\n작사\nEgg3\n작사\nEgg1\n작곡\nEgg2\n...,"95,206",3,그룹,남성,데뷔곡\nLa La La\n스탠딩 에그


In [8]:
# 크롬 드라이버 열기
driver = webdriver.Chrome('chromedriver.exe')
driver.implicitly_wait(5) # 대기 걸기
driver.maximize_window()

# 각 컬럼별 데이터 리스트 선정 (15개)

year = [] # 연도
rank = [] # 순위
title = [] # 곡 명
singer= [] # 가수
album = [] # 앨범명
like_count = [] # 좋아요 수

date_of_issue = [] # 앨범 발매일
genre = [] # 장르
lyrics = [] # 가사
three = [] # 작사/작곡/편곡

fan_count = [] # 팬 수

member_count = [] # 멤버수
solor_group = [] # 솔로/그룹여부
gender = [] # 성별
debut_song = [] # 데뷔곡 명


# 51~100위곡 순위, 곡 제목, 가수, 앨범, 좋아요 수 수집
for i in range(50, 100):
    # 드라이버가 해당 url 접속
    url = 'https://www.melon.com/chart/month/index.htm' # 멜론 월간 차트 url
    driver.get(url)


    # 국내 종합 클릭
    driver.find_element_by_xpath('/html/body/div/div[3]/div/div/div[2]/dl/dd[1]/ul/li[2]/a/span').click()


    # 월 고르기 버튼 클릭
    driver.find_element_by_xpath('/html/body/div/div[3]/div/div/div[3]/div/button').click()
    time.sleep(1)

    # 1월 클릭
    driver.find_element_by_xpath('/html/body/div/div[3]/div/div/div[3]/div/div/dl/dd[1]/ul/li[1]/a/span').click()
    time.sleep(1)
    
    year.append('2021.01')
    rank.append(driver.find_element_by_xpath(f'/html/body/div/div[3]/div/div/div[4]/form/div/table/tbody/tr[{i+1}]/td[2]/div/span[1]').text)
    title.append(driver.find_element_by_xpath(f'/html/body/div/div[3]/div/div/div[4]/form/div/table/tbody/tr[{i+1}]/td[6]/div/div/div[1]/span/a').text)
    singer.append(driver.find_element_by_xpath(f'/html/body/div/div[3]/div/div/div[4]/form/div/table/tbody/tr[{i+1}]/td[6]/div/div/div[2]/a[1]').text)
    album.append(driver.find_element_by_xpath(f'/html/body/div/div[3]/div/div/div[4]/form/div/table/tbody/tr[{i+1}]/td[7]/div/div/div/a').text)
    like_count.append(driver.find_element_by_xpath(f'/html/body/div/div[3]/div/div/div[4]/form/div/table/tbody/tr[{i+1}]/td[8]/div/button/span[2]').text)
    
    

    # 앨범명 클릭 페이지 (발매일, 장르, 가사, 작사/작곡/편곡)
    driver.find_element_by_xpath(f'/html/body/div/div[3]/div/div/div[4]/form/div/table/tbody/tr[{i+1}]/td[5]/div/a').click()
    time.sleep(2)
    date_of_issue.append(driver.find_element_by_xpath(f'/html/body/div[1]/div[3]/div/div/div/form/div/div/div[2]/div[2]/dl/dd[2]').text)
    genre.append(driver.find_element_by_xpath(f'/html/body/div[1]/div[3]/div/div/div/form/div/div/div[2]/div[2]/dl/dd[3]').text)
    lyrics.append(driver.find_element_by_xpath(f'/html/body/div[1]/div[3]/div/div/div/div[2]/div[2]/div').text)


    temp = driver.find_elements_by_xpath('/html/body/div[1]/div[3]/div/div/div/div[3]/ul')
    time.sleep(2)
    for i in range(len(temp)):
        three.append(temp[i].text)

    # 가수명 페이지 클릭 (팬 수를 수집하기 위함)
    driver.find_element_by_xpath('/html/body/div[1]/div[3]/div/div/div/form/div/div/div[2]/div[1]/div[2]/a/span[1]').click()
    time.sleep(2)
    fan_count.append(driver.find_element_by_id('d_like_count').text)

    # 상세정보 페이지 클릭 (멤버수,솔로/그룹,성별,데뷔곡을 수집하기 위함)
    try:
        driver.find_element_by_xpath('/html/body/div/div[3]/div/div/div[3]/ul/li[2]/a').click()
        time.sleep(2)
    except:
        driver.find_element_by_xpath('/html/body/div/div[3]/div/div/div[4]/ul/li[2]/a').click()
        time.sleep(2)

    try:
        member_count.append(len(driver.find_element_by_class_name('wrap_atistname').text.split(',')))
        solor_group.append("그룹")

    except:
        member_count.append(1)
        solor_group.append("솔로")

    mfm = driver.find_element_by_class_name('section_atistinfo03').text
    time.sleep(2)
    if mfm.find("혼성") >= 0:
        gender.append("혼성")
    elif mfm.find("남성") >= 0:
        gender.append("남성")
    else:
        gender.append("여성")

    try:
        debut_song.append(driver.find_element_by_class_name('debutsong').text)

    except:
        debut_song.append('')


In [9]:

#수집한 데이터들을 각 컬럼별로 리스트에 담아서 데이터프레임으로 만들기
df2 = pd.DataFrame({'연도' : year,
                   '순위': rank,
                   '제목': title,
                   '가수' : singer,
                   '앨범' : album,
                   '좋아요 수' : like_count,
                   '발매일' : date_of_issue,
                   '장르' : genre,
                   '가사' : lyrics,
                   '작사/작곡/편곡' : three,
                   '팬수' : fan_count,
                   '멤버수' : member_count,
                   '솔로/그룹' : solor_group,
                   '성별' : gender,
                   '데뷔곡' : debut_song
                  })
df2

,연도,순위,제목,가수,앨범,좋아요 수,발매일,장르,가사,작사/작곡/편곡,팬수,멤버수,솔로/그룹,성별,데뷔곡
0,2021.01,51,한잔이면 지워질까,황인욱,한잔이면 지워질까,"24,220",2020.12.15,발라드,한 잔이면 지워질까\n취할수록 선명해져\n우리 되돌릴 수 없단 거 알아\n너와 자주...,황인욱\n작사\n황인욱\n작곡\n전태익\n작곡\n전태익\n편곡\n이래언\n편곡,"9,513",1,솔로,남성,데뷔곡\n취하고 싶다 (Prod. By 전태익)\n황인욱
1,2021.01,52,뻔한남자,이승기,뻔한남자,"42,943",2020.11.15,발라드,그 날 이후 두 사람\n어느 정도 예감했지\n멀지 않은 우리 이별의 날을\n달라지는...,윤종신\n작사\n윤종신\n작곡\n나원주\n편곡,"32,506",1,솔로,남성,데뷔곡\n내 여자라니까\n이승기
2,2021.01,53,눈누난나 (NUNU NANA),제시 (Jessi),NUNA,"102,053",2020.07.30,랩/힙합,I’m trying to give u\nsomething more\nSo come ...,제시 (Jessi)\n작사\n싸이 (PSY)\n작사\nILLSON\n작사\n페노메코...,"29,107",1,솔로,여성,데뷔곡\nGet Up\n제시 (Jessi)
3,2021.01,54,Downtown Baby,블루 (BLOO),Downtown Baby,"189,330",2017.12.06,"랩/힙합, 인디음악",너는 나의 다운타운 베이비야\n너의 눈은 밤하늘에 별이야\n매일마다 꾸고 싶은 꿈이...,블루 (BLOO)\n작사\nROCK IT PRODUCTIONS\n작곡\n블루 (BL...,"32,664",1,솔로,남성,데뷔곡\nDrive Thru\n블루 (BLOO)
4,2021.01,55,가을밤에 든 생각,잔나비,잔나비 소곡집 l,"77,422",2020.11.06,"인디음악, 록/메탈",머나먼 별빛 저 별에서도\n노랠 부르는 사랑 살겠지\n밤이면 오손도손 그리운 것들 ...,잔나비 최정훈\n작사\n잔나비 최정훈\n작곡\n김도형\n작곡\n잔나비 최정훈\n편곡...,"110,842",4,그룹,남성,데뷔곡\n로켓트\n잔나비
5,2021.01,56,안녕,폴킴,호텔 델루나 OST Part.10,"193,439",2019.08.12,"발라드, 국내드라마",안녕 이라는 말을해\n짧은 시간을 뒤로 한채로\n여전히 아프겠지만\n하룻밤 자고 나...,폴킴\n작사\n지훈\n작사\n박세준\n작사\n로코베리\n작곡\n폴킴\n작곡\n로코베...,"151,296",1,솔로,남성,데뷔곡\n커피 한 잔 할래요\n폴킴
6,2021.01,57,"사랑 못해, 남들 쉽게 다 하는 거",먼데이 키즈 (Monday Kiz),"사랑 못해, 남들 쉽게 다 하는 거","35,236",2020.10.21,발라드,따스해진 날에 너를 생각해\n넌 뭐하고 지내 별일 없는지\n그렇게 말없이 떠나가고서...,한상원\n작사\n이진성\n작사\n한상원\n작곡\n이진성\n작곡\n한상원\n편곡\n이...,"38,773",1,솔로,남성,데뷔곡\nBye Bye Bye\n먼데이 키즈 (Monday Kiz)
7,2021.01,58,Panorama,IZ*ONE (아이즈원),One-reeler / Act IV,"69,414",2020.12.07,댄스,Shoot! Take a Panorama\n\n바람 끝에 찾아온 작은 기억\n귓가에...,Kz\n작사\n비오 (B.O.)\n작사\nFAB\n작사\nKz\n작곡\nNthoni...,"85,893",12,그룹,여성,데뷔곡\n라비앙로즈 (La Vie en Rose)\nIZ*ONE (아이즈원)
8,2021.01,59,고독하구만 (Feat. 수퍼비) (Prod. GroovyRoom),머쉬베놈 (MUSHVENOM),쇼미더머니 9 Semi Final,"31,842",2020.12.12,랩/힙합,고독하구만\nGo독 Go독 Go독 고독하구만\nGo독 Go독 Go독 고독하구만\n참...,머쉬베놈 (MUSHVENOM)\n작사\n수퍼비\n작사\n그루비룸 (GroovyRoo...,"23,508",1,솔로,남성,
9,2021.01,60,너를 만나,폴킴,너를 만나,"291,166",2018.10.29,발라드,너를 만난 그 이후로\n사소한 변화들에 행복해져\n눈이 부시게 빛 나는 아침\n너를...,폴킴\n작사\n폴킴\n작곡\n도니제이\n작곡\n조셉 케이\n작곡\n조셉 케이\n편곡...,"151,296",1,솔로,남성,데뷔곡\n커피 한 잔 할래요\n폴킴


In [13]:
result_1 = pd.concat([df1, df2], ignore_index=True)
result_1

,연도,순위,제목,가수,앨범,좋아요 수,발매일,장르,가사,작사/작곡/편곡,팬수,멤버수,솔로/그룹,성별,데뷔곡
0,2021.01,1,VVS (Feat. JUSTHIS) (Prod. GroovyRoom),미란이 (Mirani),쇼미더머니 9 Episode 1,"187,269",2020.11.21,랩/힙합,보여줘야겠어\n내가 망할 거랬던 놈들에게도\n내가 잘될 거라 했던 너에게도\n다 할...,저스디스 (JUSTHIS)\n작사\n그루비룸 (GroovyRoom)\n작사\n식케이...,"14,384",1,솔로,여성,
1,2021.01,2,밤하늘의 별을(2020),경서,밤하늘의 별을(2020),"157,568",2020.11.14,발라드,밤하늘의 별을 따서 너에게 줄래\n너는 내가 사랑하니까\n더 소중하니까\n오직 너 ...,강봄\n작사\n강봄\n작곡\n양정승\n편곡,"4,433",1,솔로,여성,
2,2021.01,3,Dynamite,방탄소년단,Dynamite (DayTime Version),"418,048",2020.08.24,댄스,Cos ah ah\nI’m in the stars tonight\nSo watch ...,David Stewart\n작사\nJessica Agombar\n작사\nDavid ...,"778,049",7,그룹,남성,데뷔곡\nNo More Dream\n방탄소년단
3,2021.01,4,잠이 오질 않네요,장범준,잠이 오질 않네요,"143,230",2020.10.24,발라드,당신은 날 설레게 만들어\n조용한 내 마음 자꾸만 춤추게 해\n얼마나 얼마나 날 떨...,장범준\n작사\n박경구\n작사\n장범준\n작곡\n장범준\n편곡\n전영호\n편곡\n황...,"103,946",1,솔로,남성,데뷔곡\n어려운 여자\n장범준
4,2021.01,5,Lovesick Girls,BLACKPINK,THE ALBUM,"194,970",2020.10.02,댄스,영원한 밤\n창문 없는 방에 우릴 가둔 love\nWhat can we say\n매...,TEDDY\n작사\nLØREN\n작사\n지수 (JISOO)\n작사\n제니 (JENN...,"209,935",4,그룹,여성,데뷔곡\n휘파람\nBLACKPINK
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,2021.01,96,밤편지,아이유,밤편지,"419,159",2017.03.24,발라드,이 밤 그날의 반딧불을\n당신의 창 가까이 보낼게요\n음 사랑한다는 말이에요\n나 ...,아이유\n작사\n제휘\n작곡\n김희원\n작곡\n제휘\n편곡\n김희원\n편곡,"746,234",1,솔로,여성,데뷔곡\n미아\n아이유
96,2021.01,97,크리스마스니까,성시경,Jelly Christmas 2012 HEART PROJECT,"159,133",2012.12.06,발라드,오늘은 뭔가 좀 다른 기분 Feeling\n이젠 때가 된 것 같은 느낌 Love U...,서정진\n작사\n김두현\n작사\n서정진\n작곡\n김두현\n작곡\n서정진\n편곡\n김...,"117,219",1,솔로,남성,데뷔곡\n내게 오는 길\n성시경
97,2021.01,98,첫눈처럼 너에게 가겠다,에일리(AILEE),도깨비 OST Part.9,"326,257",2017.01.07,"발라드, 국내드라마",널 품기 전 알지 못했다\n내 머문 세상 이토록 찬란한 것을\n작은 숨결로 닿은 사...,이미나\n작사\n로코베리\n작곡\n로코베리\n편곡,"117,796",1,솔로,여성,데뷔곡\nHeaven\n에일리(AILEE)
98,2021.01,99,Pretty Savage,BLACKPINK,THE ALBUM,"68,943",2020.10.02,랩/힙합,비슷한 것 같지 우린 뼛속까지 다름\n아이 창피하다가도 멍석 깔면 바름\nBorn ...,TEDDY\n작사\nLØREN\n작사\nVince\n작사\nDanny Chung\n...,"209,943",4,그룹,여성,데뷔곡\n휘파람\nBLACKPINK


In [28]:
# csv 파일로 저장
result_1.to_csv('Melon_2021_01.csv',encoding='utf-8-sig')

# 2021.02월 데이터 수집

In [15]:
############### 2월 데이터 수집

# 크롬 드라이버 열기
driver = webdriver.Chrome('chromedriver.exe')
driver.implicitly_wait(5) # 대기 걸기
driver.maximize_window()
    
# 크롤링으로 데이터 수집

# 각 컬럼별 데이터 리스트 선정 (15개)

year = [] # 연도
rank = [] # 순위
title = [] # 곡 명
singer= [] # 가수
album = [] # 앨범명
like_count = [] # 좋아요 수

date_of_issue = [] # 앨범 발매일
genre = [] # 장르
lyrics = [] # 가사
three = [] # 작사/작곡/편곡

fan_count = [] # 팬 수

member_count = [] # 멤버수
solor_group = [] # 솔로/그룹여부
gender = [] # 성별
debut_song = [] # 데뷔곡 명


# 1~50위곡 순위, 곡 제목, 가수, 앨범, 좋아요 수 수집
for i in range(50):
    # 드라이버가 해당 url 접속
    url = 'https://www.melon.com/chart/month/index.htm' # 멜론 월간 차트 url
    driver.get(url)


    # 국내 종합 클릭
    driver.find_element_by_xpath('/html/body/div/div[3]/div/div/div[2]/dl/dd[1]/ul/li[2]/a/span').click()


    # 월 고르기 버튼 클릭
    driver.find_element_by_xpath('/html/body/div/div[3]/div/div/div[3]/div/button').click()
    time.sleep(1)

    # 2월 클릭
    driver.find_element_by_xpath('/html/body/div/div[3]/div/div/div[3]/div/div/dl/dd[1]/ul/li[2]/a/span').click()
    time.sleep(1)
    
    year.append('2021.02')
    rank.append(driver.find_element_by_xpath(f'/html/body/div/div[3]/div/div/div[4]/form/div/table/tbody/tr[{i+1}]/td[2]/div/span[1]').text)
    title.append(driver.find_element_by_xpath(f'/html/body/div/div[3]/div/div/div[4]/form/div/table/tbody/tr[{i+1}]/td[6]/div/div/div[1]/span/a').text)
    singer.append(driver.find_element_by_xpath(f'/html/body/div/div[3]/div/div/div[4]/form/div/table/tbody/tr[{i+1}]/td[6]/div/div/div[2]/a[1]').text)
    album.append(driver.find_element_by_xpath(f'/html/body/div/div[3]/div/div/div[4]/form/div/table/tbody/tr[{i+1}]/td[7]/div/div/div/a').text)
    like_count.append(driver.find_element_by_xpath(f'/html/body/div/div[3]/div/div/div[4]/form/div/table/tbody/tr[{i+1}]/td[8]/div/button/span[2]').text)
    
    

    # 앨범명 클릭 페이지 (발매일, 장르, 가사, 작사/작곡/편곡)
    driver.find_element_by_xpath(f'/html/body/div/div[3]/div/div/div[4]/form/div/table/tbody/tr[{i+1}]/td[5]/div/a').click()
    time.sleep(2)
    date_of_issue.append(driver.find_element_by_xpath(f'/html/body/div[1]/div[3]/div/div/div/form/div/div/div[2]/div[2]/dl/dd[2]').text)
    genre.append(driver.find_element_by_xpath(f'/html/body/div[1]/div[3]/div/div/div/form/div/div/div[2]/div[2]/dl/dd[3]').text)
    lyrics.append(driver.find_element_by_xpath(f'/html/body/div[1]/div[3]/div/div/div/div[2]/div[2]/div').text)


    temp = driver.find_elements_by_xpath('/html/body/div[1]/div[3]/div/div/div/div[3]/ul')
    time.sleep(2)
    for i in range(len(temp)):
        three.append(temp[i].text)

    # 가수명 페이지 클릭 (팬 수를 수집하기 위함)
    driver.find_element_by_xpath('/html/body/div[1]/div[3]/div/div/div/form/div/div/div[2]/div[1]/div[2]/a/span[1]').click()
    time.sleep(2)
    fan_count.append(driver.find_element_by_id('d_like_count').text)

    # 상세정보 페이지 클릭 (멤버수,솔로/그룹,성별,데뷔곡을 수집하기 위함)
    try:
        driver.find_element_by_xpath('/html/body/div/div[3]/div/div/div[3]/ul/li[2]/a').click()
        time.sleep(2)
    except:
        driver.find_element_by_xpath('/html/body/div/div[3]/div/div/div[4]/ul/li[2]/a').click()
        time.sleep(2)

    try:
        member_count.append(len(driver.find_element_by_class_name('wrap_atistname').text.split(',')))
        solor_group.append("그룹")

    except:
        member_count.append(1)
        solor_group.append("솔로")

    mfm = driver.find_element_by_class_name('section_atistinfo03').text
    time.sleep(2)
    if mfm.find("혼성") >= 0:
        gender.append("혼성")
    elif mfm.find("남성") >= 0:
        gender.append("남성")
    else:
        gender.append("여성")

    try:
        debut_song.append(driver.find_element_by_class_name('debutsong').text)

    except:
        debut_song.append('')

In [16]:
#수집한 데이터들을 각 컬럼별로 리스트에 담아서 데이터프레임으로 만들기
df3 = pd.DataFrame({'연도' : year,
                   '순위': rank,
                   '제목': title,
                   '가수' : singer,
                   '앨범' : album,
                   '좋아요 수' : like_count,
                   '발매일' : date_of_issue,
                   '장르' : genre,
                   '가사' : lyrics,
                   '작사/작곡/편곡' : three,
                   '팬수' : fan_count,
                   '멤버수' : member_count,
                   '솔로/그룹' : solor_group,
                   '성별' : gender,
                   '데뷔곡' : debut_song
                  })
df3

,연도,순위,제목,가수,앨범,좋아요 수,발매일,장르,가사,작사/작곡/편곡,팬수,멤버수,솔로/그룹,성별,데뷔곡
0,2021.02,1,Celebrity,아이유,IU 5th Album 'LILAC',"293,000",2021.03.25,댄스,"세상의 모서리\n구부정하게 커버린\n골칫거리 outsider\n\n걸음걸이, 옷차림...",아이유\n작사\nRyan S. Jhun\n작곡\nJeppe London Bilsby...,"746,238",1,솔로,여성,데뷔곡\n미아\n아이유
1,2021.02,2,밤하늘의 별을(2020),경서,밤하늘의 별을(2020),"157,577",2020.11.14,발라드,밤하늘의 별을 따서 너에게 줄래\n너는 내가 사랑하니까\n더 소중하니까\n오직 너 ...,강봄\n작사\n강봄\n작곡\n양정승\n편곡,"4,431",1,솔로,여성,
2,2021.02,3,Dynamite,방탄소년단,Dynamite (DayTime Version),"418,065",2020.08.24,댄스,Cos ah ah\nI’m in the stars tonight\nSo watch ...,David Stewart\n작사\nJessica Agombar\n작사\nDavid ...,"778,067",7,그룹,남성,데뷔곡\nNo More Dream\n방탄소년단
3,2021.02,4,VVS (Feat. JUSTHIS) (Prod. GroovyRoom),미란이 (Mirani),쇼미더머니 9 Episode 1,"187,273",2020.11.21,랩/힙합,보여줘야겠어\n내가 망할 거랬던 놈들에게도\n내가 잘될 거라 했던 너에게도\n다 할...,저스디스 (JUSTHIS)\n작사\n그루비룸 (GroovyRoom)\n작사\n식케이...,"14,386",1,솔로,여성,
4,2021.02,5,잠이 오질 않네요,장범준,잠이 오질 않네요,"143,235",2020.10.24,발라드,당신은 날 설레게 만들어\n조용한 내 마음 자꾸만 춤추게 해\n얼마나 얼마나 날 떨...,장범준\n작사\n박경구\n작사\n장범준\n작곡\n장범준\n편곡\n전영호\n편곡\n황...,"103,948",1,솔로,남성,데뷔곡\n어려운 여자\n장범준
5,2021.02,6,Lovesick Girls,BLACKPINK,THE ALBUM,"194,974",2020.10.02,댄스,영원한 밤\n창문 없는 방에 우릴 가둔 love\nWhat can we say\n매...,TEDDY\n작사\nLØREN\n작사\n지수 (JISOO)\n작사\n제니 (JENN...,"209,946",4,그룹,여성,데뷔곡\n휘파람\nBLACKPINK
6,2021.02,7,이 밤을 빌려 말해요 (바른연애 길잡이 X 10CM),10CM,이 밤을 빌려 말해요 (바른연애 길잡이 X 10CM),"71,117",2021.01.24,발라드,나의 두 눈에 제일 먼저 그대를 담고\n가장 예쁜 말들로 그댈 미소 짓게 하고\n바...,윤토벤\n작사\n손박사\n작사\nI.AN\n작사\n손박사\n작곡\n윤토벤\n작곡\n...,"157,005",1,솔로,남성,데뷔곡\nGood Night\n10CM
7,2021.02,8,취기를 빌려 (취향저격 그녀 X 산들),산들,취기를 빌려 (취향저격 그녀 X 산들),"195,854",2020.07.20,발라드,언제부턴가 불쑥\n내 습관이 돼버린 너\n혹시나 이런 맘이\n어쩌면 부담일까\n널 ...,새봄 (saevom)\n작사\n새봄 (saevom)\n작곡\n박민주\n편곡,"27,249",1,솔로,남성,데뷔곡\nO.K\n산들
8,2021.02,9,에잇(Prod.&Feat. SUGA of BTS),아이유,에잇,"363,088",2020.05.06,록/메탈,So are you happy now\nFinally happy now are yo...,아이유\n작사\nSUGA\n작사\nSUGA\n작곡\n아이유\n작곡\nEL CAPIT...,"746,238",1,솔로,여성,데뷔곡\n미아\n아이유
9,2021.02,10,힘든 건 사랑이 아니다,임창정,힘든 건 사랑이 아니다,"106,207",2020.10.19,발라드,내가 널 떠났어야 했는데 왜 떠나야 하는지도 아는데\n어떤 아무 말도 아무것도 줄 ...,임창정\n작사\n임창정\n작곡\n멧돼지\n작곡\n늑대\n작곡\n멧돼지\n편곡\n늑대...,"95,458",1,솔로,남성,


In [18]:
# 크롬 드라이버 열기
driver = webdriver.Chrome('chromedriver.exe')
driver.implicitly_wait(5) # 대기 걸기
driver.maximize_window()
    
# 크롤링으로 데이터 수집

# 각 컬럼별 데이터 리스트 선정 (15개)

year = [] # 연도
rank = [] # 순위
title = [] # 곡 명
singer= [] # 가수
album = [] # 앨범명
like_count = [] # 좋아요 수

date_of_issue = [] # 앨범 발매일
genre = [] # 장르
lyrics = [] # 가사
three = [] # 작사/작곡/편곡

fan_count = [] # 팬 수

member_count = [] # 멤버수
solor_group = [] # 솔로/그룹여부
gender = [] # 성별
debut_song = [] # 데뷔곡 명


# 51~100위곡 순위, 곡 제목, 가수, 앨범, 좋아요 수 수집
for i in range(50,100):
    # 드라이버가 해당 url 접속
    url = 'https://www.melon.com/chart/month/index.htm' # 멜론 월간 차트 url
    driver.get(url)


    # 국내 종합 클릭
    driver.find_element_by_xpath('/html/body/div/div[3]/div/div/div[2]/dl/dd[1]/ul/li[2]/a/span').click()


    # 월 고르기 버튼 클릭
    driver.find_element_by_xpath('/html/body/div/div[3]/div/div/div[3]/div/button').click()
    time.sleep(1)

    # 2월 클릭
    driver.find_element_by_xpath('/html/body/div/div[3]/div/div/div[3]/div/div/dl/dd[1]/ul/li[2]/a/span').click()
    time.sleep(1)
    
    year.append('2021.02')
    rank.append(driver.find_element_by_xpath(f'/html/body/div/div[3]/div/div/div[4]/form/div/table/tbody/tr[{i+1}]/td[2]/div/span[1]').text)
    title.append(driver.find_element_by_xpath(f'/html/body/div/div[3]/div/div/div[4]/form/div/table/tbody/tr[{i+1}]/td[6]/div/div/div[1]/span/a').text)
    singer.append(driver.find_element_by_xpath(f'/html/body/div/div[3]/div/div/div[4]/form/div/table/tbody/tr[{i+1}]/td[6]/div/div/div[2]/a[1]').text)
    album.append(driver.find_element_by_xpath(f'/html/body/div/div[3]/div/div/div[4]/form/div/table/tbody/tr[{i+1}]/td[7]/div/div/div/a').text)
    like_count.append(driver.find_element_by_xpath(f'/html/body/div/div[3]/div/div/div[4]/form/div/table/tbody/tr[{i+1}]/td[8]/div/button/span[2]').text)
    
    

    # 앨범명 클릭 페이지 (발매일, 장르, 가사, 작사/작곡/편곡)
    driver.find_element_by_xpath(f'/html/body/div/div[3]/div/div/div[4]/form/div/table/tbody/tr[{i+1}]/td[5]/div/a').click()
    time.sleep(2)
    date_of_issue.append(driver.find_element_by_xpath(f'/html/body/div[1]/div[3]/div/div/div/form/div/div/div[2]/div[2]/dl/dd[2]').text)
    genre.append(driver.find_element_by_xpath(f'/html/body/div[1]/div[3]/div/div/div/form/div/div/div[2]/div[2]/dl/dd[3]').text)
    lyrics.append(driver.find_element_by_xpath(f'/html/body/div[1]/div[3]/div/div/div/div[2]/div[2]/div').text)


    temp = driver.find_elements_by_xpath('/html/body/div[1]/div[3]/div/div/div/div[3]/ul')
    time.sleep(2)
    for i in range(len(temp)):
        three.append(temp[i].text)

    # 가수명 페이지 클릭 (팬 수를 수집하기 위함)
    driver.find_element_by_xpath('/html/body/div[1]/div[3]/div/div/div/form/div/div/div[2]/div[1]/div[2]/a/span[1]').click()
    time.sleep(2)
    fan_count.append(driver.find_element_by_id('d_like_count').text)

    # 상세정보 페이지 클릭 (멤버수,솔로/그룹,성별,데뷔곡을 수집하기 위함)
    try:
        driver.find_element_by_xpath('/html/body/div/div[3]/div/div/div[3]/ul/li[2]/a').click()
        time.sleep(2)
    except:
        driver.find_element_by_xpath('/html/body/div/div[3]/div/div/div[4]/ul/li[2]/a').click()
        time.sleep(2)

    try:
        member_count.append(len(driver.find_element_by_class_name('wrap_atistname').text.split(',')))
        solor_group.append("그룹")

    except:
        member_count.append(1)
        solor_group.append("솔로")

    mfm = driver.find_element_by_class_name('section_atistinfo03').text
    time.sleep(2)
    if mfm.find("혼성") >= 0:
        gender.append("혼성")
    elif mfm.find("남성") >= 0:
        gender.append("남성")
    else:
        gender.append("여성")

    try:
        debut_song.append(driver.find_element_by_class_name('debutsong').text)

    except:
        debut_song.append('')

In [19]:
#수집한 데이터들을 각 컬럼별로 리스트에 담아서 데이터프레임으로 만들기
df4 = pd.DataFrame({'연도' : year,
                   '순위': rank,
                   '제목': title,
                   '가수' : singer,
                   '앨범' : album,
                   '좋아요 수' : like_count,
                   '발매일' : date_of_issue,
                   '장르' : genre,
                   '가사' : lyrics,
                   '작사/작곡/편곡' : three,
                   '팬수' : fan_count,
                   '멤버수' : member_count,
                   '솔로/그룹' : solor_group,
                   '성별' : gender,
                   '데뷔곡' : debut_song
                  })
df4

,연도,순위,제목,가수,앨범,좋아요 수,발매일,장르,가사,작사/작곡/편곡,팬수,멤버수,솔로/그룹,성별,데뷔곡
0,2021.02,51,I CAN’T STOP ME,TWICE (트와이스),Eyes wide open,"63,839",2020.10.26,댄스,알람이 울려대 Ring ring a ling\n서로의 눈길이 닿을 때마다\n알면서 ...,J.Y. Park `The Asiansoul`\n작사\n심은지\n작사\nMelani...,"215,924",9,그룹,여성,데뷔곡\nOOH-AHH하게\nTWICE (트와이스)
1,2021.02,52,사랑은 지날수록 더욱 선명하게 남아,전상근,한 걸음 : 흔적,"66,172",2020.06.07,발라드,편해지고 변해가고\n자연스러운 그런 과정\n시도 때도 없이 다투면서\n불안해진 마음...,빅가이로빈 (bigguyrobin)\n작사\n빅가이로빈 (bigguyrobin)\n...,"11,599",1,솔로,남성,
2,2021.02,53,"Achoo (Feat. pH-1, HAON) (Prod. GroovyRoom)",미란이 (Mirani),쇼미더머니 9 Episode 3,"61,973",2020.12.05,랩/힙합,바뀔 시간이 됐어 We go up with party\n벌어 Cash fame 이젠...,미란이 (Mirani)\n작사\npH-1\n작사\n김하온 (HAON)\n작사\n그루...,"14,386",1,솔로,여성,
3,2021.02,54,DON'T TOUCH ME,환불원정대,DON'T TOUCH ME,"160,186",2020.10.10,댄스,Trouble 이래 다 그래\n세 보인대 어쩔래\n난 멋 부리네 더 꾸미네\nyea...,블랙아이드 필승\n작사\n전군\n작사\n은비\n작사\nILLSON\n작사\nJohn...,"16,217",4,그룹,여성,
4,2021.02,55,추억은 만남보다 이별에 남아,정동하,추억은 만남보다 이별에 남아,"50,949",2021.01.31,발라드,아득한 시간 속에 아직\n우리 사랑이 남아있을까\n멀어지던 그날의 너를 따라 걸어도...,문성욱\n작사\n문성욱\n작곡\n방민규\n작곡\n문성욱\n편곡\n정혜원\n편곡,"11,446",1,솔로,남성,
5,2021.02,56,Panorama,IZ*ONE (아이즈원),One-reeler / Act IV,"69,417",2020.12.07,댄스,Shoot! Take a Panorama\n\n바람 끝에 찾아온 작은 기억\n귓가에...,Kz\n작사\n비오 (B.O.)\n작사\nFAB\n작사\nKz\n작곡\nNthoni...,"85,895",12,그룹,여성,데뷔곡\n라비앙로즈 (La Vie en Rose)\nIZ*ONE (아이즈원)
6,2021.02,57,Downtown Baby,블루 (BLOO),Downtown Baby,"189,331",2017.12.06,"랩/힙합, 인디음악",너는 나의 다운타운 베이비야\n너의 눈은 밤하늘에 별이야\n매일마다 꾸고 싶은 꿈이...,블루 (BLOO)\n작사\nROCK IT PRODUCTIONS\n작곡\n블루 (BL...,"32,664",1,솔로,남성,데뷔곡\nDrive Thru\n블루 (BLOO)
7,2021.02,58,안녕,폴킴,호텔 델루나 OST Part.10,"193,441",2019.08.12,"발라드, 국내드라마",안녕 이라는 말을해\n짧은 시간을 뒤로 한채로\n여전히 아프겠지만\n하룻밤 자고 나...,폴킴\n작사\n지훈\n작사\n박세준\n작사\n로코베리\n작곡\n폴킴\n작곡\n로코베...,"151,299",1,솔로,남성,데뷔곡\n커피 한 잔 할래요\n폴킴
8,2021.02,59,우린 어쩌다 헤어진 걸까,허각,Hello,"36,430",2020.12.28,발라드,모두 처음이었어 설레어 잠 못 든 것\n내겐 처음이었어 소리 내 울었던 것도\n이미...,양재선\n작사\n빅가이로빈 (bigguyrobin)\n작곡\n최상언\n편곡\n김홍준...,"58,976",1,솔로,남성,데뷔곡\n언제나\n허각
9,2021.02,60,홀로,이하이,홀로,"136,876",2020.07.23,R&B/Soul,홀로 있는 게 가만히 있는 게\n어려운 일인가요\n홀로 있어도 같이 있어도\n외로운...,안신애\n작사\n안신애\n작곡\nglowingdog (글로잉독)\n편곡,"69,904",1,솔로,여성,"데뷔곡\n1,2,3,4 (원,투,쓰리,포)\n이하이"


In [29]:
# 2월 데이터 프레임 합치기
result_2 = pd.concat([df3, df4], ignore_index=True)
result_2

,연도,순위,제목,가수,앨범,좋아요 수,발매일,장르,가사,작사/작곡/편곡,팬수,멤버수,솔로/그룹,성별,데뷔곡
0,2021.02,1,Celebrity,아이유,IU 5th Album 'LILAC',"293,000",2021.03.25,댄스,"세상의 모서리\n구부정하게 커버린\n골칫거리 outsider\n\n걸음걸이, 옷차림...",아이유\n작사\nRyan S. Jhun\n작곡\nJeppe London Bilsby...,"746,238",1,솔로,여성,데뷔곡\n미아\n아이유
1,2021.02,2,밤하늘의 별을(2020),경서,밤하늘의 별을(2020),"157,577",2020.11.14,발라드,밤하늘의 별을 따서 너에게 줄래\n너는 내가 사랑하니까\n더 소중하니까\n오직 너 ...,강봄\n작사\n강봄\n작곡\n양정승\n편곡,"4,431",1,솔로,여성,
2,2021.02,3,Dynamite,방탄소년단,Dynamite (DayTime Version),"418,065",2020.08.24,댄스,Cos ah ah\nI’m in the stars tonight\nSo watch ...,David Stewart\n작사\nJessica Agombar\n작사\nDavid ...,"778,067",7,그룹,남성,데뷔곡\nNo More Dream\n방탄소년단
3,2021.02,4,VVS (Feat. JUSTHIS) (Prod. GroovyRoom),미란이 (Mirani),쇼미더머니 9 Episode 1,"187,273",2020.11.21,랩/힙합,보여줘야겠어\n내가 망할 거랬던 놈들에게도\n내가 잘될 거라 했던 너에게도\n다 할...,저스디스 (JUSTHIS)\n작사\n그루비룸 (GroovyRoom)\n작사\n식케이...,"14,386",1,솔로,여성,
4,2021.02,5,잠이 오질 않네요,장범준,잠이 오질 않네요,"143,235",2020.10.24,발라드,당신은 날 설레게 만들어\n조용한 내 마음 자꾸만 춤추게 해\n얼마나 얼마나 날 떨...,장범준\n작사\n박경구\n작사\n장범준\n작곡\n장범준\n편곡\n전영호\n편곡\n황...,"103,948",1,솔로,남성,데뷔곡\n어려운 여자\n장범준
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,2021.02,96,주저하는 연인들을 위해,잔나비,전설,"295,095",2019.03.13,"인디음악, 록/메탈",나는 읽기 쉬운 마음이야\n당신도 스윽 훑고 가셔요\n달랠 길 없는 외로운 마음 있...,잔나비 최정훈\n작사\n잔나비 최정훈\n작곡\n김도형\n작곡\n유영현\n작곡\n잔나...,"110,851",4,그룹,남성,데뷔곡\n로켓트\n잔나비
96,2021.02,97,그때 그 아인,김필,이태원 클라쓰 OST Part.6,"110,881",2020.02.15,"발라드, 국내드라마",길었던 하루 그림잔\n아직도 아픔을 서성일까\n말없이 기다려 보면\n쓰러질 듯 내게...,서동성\n작사\n박성일\n작곡\n박성일\n편곡,"46,973",1,솔로,남성,데뷔곡\n사랑한다면서\n김필
97,2021.02,98,"Rosario (Feat. CL, ZICO)",에픽하이 (EPIK HIGH),Epik High Is Here 上,"19,684",2021.01.18,랩/힙합,장미로 와\n가시만 남기고\n떠나가버린\n내 님아\n\n있을 때 잘해\n없을 땐 닥...,Tablo\n작사\n미쓰라 진\n작사\n지코 (ZICO)\n작사\nDJ 투컷\n작곡...,"108,716",3,그룹,남성,데뷔곡\nI Remember (Feat. Kensie)\n에픽하이 (EPIK HIGH)
98,2021.02,99,Pretty Savage,BLACKPINK,THE ALBUM,"68,952",2020.10.02,랩/힙합,비슷한 것 같지 우린 뼛속까지 다름\n아이 창피하다가도 멍석 깔면 바름\nBorn ...,TEDDY\n작사\nLØREN\n작사\nVince\n작사\nDanny Chung\n...,"209,959",4,그룹,여성,데뷔곡\n휘파람\nBLACKPINK


In [30]:
# csv 파일로 저장
result_2.to_csv('Melon_2021_02.csv',encoding='utf-8-sig')

In [31]:
# 1~2월 데이터프레임 모두 합치기
result = pd.concat([df1, df2, df3, df4], ignore_index=True)
result

,연도,순위,제목,가수,앨범,좋아요 수,발매일,장르,가사,작사/작곡/편곡,팬수,멤버수,솔로/그룹,성별,데뷔곡
0,2021.01,1,VVS (Feat. JUSTHIS) (Prod. GroovyRoom),미란이 (Mirani),쇼미더머니 9 Episode 1,"187,269",2020.11.21,랩/힙합,보여줘야겠어\n내가 망할 거랬던 놈들에게도\n내가 잘될 거라 했던 너에게도\n다 할...,저스디스 (JUSTHIS)\n작사\n그루비룸 (GroovyRoom)\n작사\n식케이...,"14,384",1,솔로,여성,
1,2021.01,2,밤하늘의 별을(2020),경서,밤하늘의 별을(2020),"157,568",2020.11.14,발라드,밤하늘의 별을 따서 너에게 줄래\n너는 내가 사랑하니까\n더 소중하니까\n오직 너 ...,강봄\n작사\n강봄\n작곡\n양정승\n편곡,"4,433",1,솔로,여성,
2,2021.01,3,Dynamite,방탄소년단,Dynamite (DayTime Version),"418,048",2020.08.24,댄스,Cos ah ah\nI’m in the stars tonight\nSo watch ...,David Stewart\n작사\nJessica Agombar\n작사\nDavid ...,"778,049",7,그룹,남성,데뷔곡\nNo More Dream\n방탄소년단
3,2021.01,4,잠이 오질 않네요,장범준,잠이 오질 않네요,"143,230",2020.10.24,발라드,당신은 날 설레게 만들어\n조용한 내 마음 자꾸만 춤추게 해\n얼마나 얼마나 날 떨...,장범준\n작사\n박경구\n작사\n장범준\n작곡\n장범준\n편곡\n전영호\n편곡\n황...,"103,946",1,솔로,남성,데뷔곡\n어려운 여자\n장범준
4,2021.01,5,Lovesick Girls,BLACKPINK,THE ALBUM,"194,970",2020.10.02,댄스,영원한 밤\n창문 없는 방에 우릴 가둔 love\nWhat can we say\n매...,TEDDY\n작사\nLØREN\n작사\n지수 (JISOO)\n작사\n제니 (JENN...,"209,935",4,그룹,여성,데뷔곡\n휘파람\nBLACKPINK
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,2021.02,96,주저하는 연인들을 위해,잔나비,전설,"295,095",2019.03.13,"인디음악, 록/메탈",나는 읽기 쉬운 마음이야\n당신도 스윽 훑고 가셔요\n달랠 길 없는 외로운 마음 있...,잔나비 최정훈\n작사\n잔나비 최정훈\n작곡\n김도형\n작곡\n유영현\n작곡\n잔나...,"110,851",4,그룹,남성,데뷔곡\n로켓트\n잔나비
196,2021.02,97,그때 그 아인,김필,이태원 클라쓰 OST Part.6,"110,881",2020.02.15,"발라드, 국내드라마",길었던 하루 그림잔\n아직도 아픔을 서성일까\n말없이 기다려 보면\n쓰러질 듯 내게...,서동성\n작사\n박성일\n작곡\n박성일\n편곡,"46,973",1,솔로,남성,데뷔곡\n사랑한다면서\n김필
197,2021.02,98,"Rosario (Feat. CL, ZICO)",에픽하이 (EPIK HIGH),Epik High Is Here 上,"19,684",2021.01.18,랩/힙합,장미로 와\n가시만 남기고\n떠나가버린\n내 님아\n\n있을 때 잘해\n없을 땐 닥...,Tablo\n작사\n미쓰라 진\n작사\n지코 (ZICO)\n작사\nDJ 투컷\n작곡...,"108,716",3,그룹,남성,데뷔곡\nI Remember (Feat. Kensie)\n에픽하이 (EPIK HIGH)
198,2021.02,99,Pretty Savage,BLACKPINK,THE ALBUM,"68,952",2020.10.02,랩/힙합,비슷한 것 같지 우린 뼛속까지 다름\n아이 창피하다가도 멍석 깔면 바름\nBorn ...,TEDDY\n작사\nLØREN\n작사\nVince\n작사\nDanny Chung\n...,"209,959",4,그룹,여성,데뷔곡\n휘파람\nBLACKPINK


In [32]:
# csv 파일로 저장
result.to_csv('Melon_2021_01~02.csv',encoding='utf-8-sig')

In [37]:
# 확인
df_tot = pd.read_csv('Melon_2021_01~02.csv', index_col=0)
df_tot

,연도,순위,제목,가수,앨범,좋아요 수,발매일,장르,가사,작사/작곡/편곡,팬수,멤버수,솔로/그룹,성별,데뷔곡
0,2021.01,1,VVS (Feat. JUSTHIS) (Prod. GroovyRoom),미란이 (Mirani),쇼미더머니 9 Episode 1,"187,269",2020.11.21,랩/힙합,보여줘야겠어\n내가 망할 거랬던 놈들에게도\n내가 잘될 거라 했던 너에게도\n다 할...,저스디스 (JUSTHIS)\n작사\n그루비룸 (GroovyRoom)\n작사\n식케이...,"14,384",1,솔로,여성,NaN
1,2021.01,2,밤하늘의 별을(2020),경서,밤하늘의 별을(2020),"157,568",2020.11.14,발라드,밤하늘의 별을 따서 너에게 줄래\n너는 내가 사랑하니까\n더 소중하니까\n오직 너 ...,강봄\n작사\n강봄\n작곡\n양정승\n편곡,"4,433",1,솔로,여성,NaN
2,2021.01,3,Dynamite,방탄소년단,Dynamite (DayTime Version),"418,048",2020.08.24,댄스,Cos ah ah\nI’m in the stars tonight\nSo watch ...,David Stewart\n작사\nJessica Agombar\n작사\nDavid ...,"778,049",7,그룹,남성,데뷔곡\nNo More Dream\n방탄소년단
3,2021.01,4,잠이 오질 않네요,장범준,잠이 오질 않네요,"143,230",2020.10.24,발라드,당신은 날 설레게 만들어\n조용한 내 마음 자꾸만 춤추게 해\n얼마나 얼마나 날 떨...,장범준\n작사\n박경구\n작사\n장범준\n작곡\n장범준\n편곡\n전영호\n편곡\n황...,"103,946",1,솔로,남성,데뷔곡\n어려운 여자\n장범준
4,2021.01,5,Lovesick Girls,BLACKPINK,THE ALBUM,"194,970",2020.10.02,댄스,영원한 밤\n창문 없는 방에 우릴 가둔 love\nWhat can we say\n매...,TEDDY\n작사\nLØREN\n작사\n지수 (JISOO)\n작사\n제니 (JENN...,"209,935",4,그룹,여성,데뷔곡\n휘파람\nBLACKPINK
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,2021.02,96,주저하는 연인들을 위해,잔나비,전설,"295,095",2019.03.13,"인디음악, 록/메탈",나는 읽기 쉬운 마음이야\n당신도 스윽 훑고 가셔요\n달랠 길 없는 외로운 마음 있...,잔나비 최정훈\n작사\n잔나비 최정훈\n작곡\n김도형\n작곡\n유영현\n작곡\n잔나...,"110,851",4,그룹,남성,데뷔곡\n로켓트\n잔나비
196,2021.02,97,그때 그 아인,김필,이태원 클라쓰 OST Part.6,"110,881",2020.02.15,"발라드, 국내드라마",길었던 하루 그림잔\n아직도 아픔을 서성일까\n말없이 기다려 보면\n쓰러질 듯 내게...,서동성\n작사\n박성일\n작곡\n박성일\n편곡,"46,973",1,솔로,남성,데뷔곡\n사랑한다면서\n김필
197,2021.02,98,"Rosario (Feat. CL, ZICO)",에픽하이 (EPIK HIGH),Epik High Is Here 上,"19,684",2021.01.18,랩/힙합,장미로 와\n가시만 남기고\n떠나가버린\n내 님아\n\n있을 때 잘해\n없을 땐 닥...,Tablo\n작사\n미쓰라 진\n작사\n지코 (ZICO)\n작사\nDJ 투컷\n작곡...,"108,716",3,그룹,남성,데뷔곡\nI Remember (Feat. Kensie)\n에픽하이 (EPIK HIGH)
198,2021.02,99,Pretty Savage,BLACKPINK,THE ALBUM,"68,952",2020.10.02,랩/힙합,비슷한 것 같지 우린 뼛속까지 다름\n아이 창피하다가도 멍석 깔면 바름\nBorn ...,TEDDY\n작사\nLØREN\n작사\nVince\n작사\nDanny Chung\n...,"209,959",4,그룹,여성,데뷔곡\n휘파람\nBLACKPINK
